In [ ]:
import sys
dlc_dir = '/Users/annateruel/phd_code/'
if dlc_dir not in sys.path:
    sys.path.append(dlc_dir)


In [ ]:
import os
import pandas as pd
import dlc.analysis.time_roi as time_roi
from dlc.analysis.time_roi import ROIDrawer
from dlc.data import Concatenate, Centroid

Concatenate your data from h5 and video files (for data recorded with Miniscope Software)

In [ ]:
dir = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/24-02-ca2+img/AD23-097/control'
concatenator = Concatenate(dir)
h5_concat= concatenator.concatenate_h5()
video_concat = concatenator.concatenate_videos('.avi')

In [ ]:
root_dir = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/24-02-ca2+img'

for subdir in os.listdir(root_dir):
    subdir_path = os.path.join(root_dir, subdir)
    if os.path.isdir(subdir_path):
        for condition in ['control', 'anosmic']:
            condition_dir = os.path.join(subdir_path, condition)
            
            if os.path.exists(condition_dir):
                concatenator = Concatenate(condition_dir)
                h5_concat = concatenator.concatenate_h5()
                video_concat = concatenator.concatenate_videos('.avi')

1. Drawing ROIS with ROIdrawer tool

In [ ]:
video = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/22-09-sr/dlc/AD21-087-HAB1_bw.mp4'
savedir = os.path.dirname(video)
roi_drawer = ROIDrawer(video_path=video,save_dir=savedir, num_rois=2)
r = roi_drawer.draw_rois()

2. Time in ROIs

Before calculating time ina region of interest, we will begin by getting the centroid of x bodyparts

In [ ]:
dir = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/22-09-sr/dlc/'
bodyparts = ["nose", "rightear", "leftear", "head", "spine1"] 

In [ ]:
centroid = Centroid(minutes = 10)
result = centroid.get_centroid(dir, bodyparts, save = True)

In [ ]:
for key, df in result.items():
        df.to_hdf(os.path.join(dir, f"{key}_centroid.h5"), key='df', mode='w')

Now we have our centoid, is time to get time in the regions of interest we have defined!

In [ ]:
video_dir = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/22-09-sr/dlc/'
scorer = 'DLC_resnet50_conductaJan11shuffle1_600000' #should take it from DLC multiindex dataframe
bodypart = 'centroid'

In [ ]:
roi_ellipse = {}
for file in os.listdir(video_dir):
    if file.endswith('roi.h5') and not file.startswith('._'):
        roi_file_path = os.path.join(video_dir, file)
        print(f"Processing ROI file: {file}")

        try:
            roi_data = pd.read_hdf(roi_file_path)
            roi_groups = roi_data.groupby(['index', 'shape-type'])
            ellipse, _ = time_roi.EllipseROI.extract_ellipses(roi_groups)
            key = file.replace('_roi.h5', '')
            roi_ellipse[key] = ellipse
            print(f"Extracted ellipses from {file}: {len(ellipse)}")
        except Exception as e:
            print(f"Error processing {file}: {e}")

In [ ]:
frames_with_zero_rois = []
for key, ellipses in roi_ellipse.items():
    if len(ellipses) == 0:
        frames_with_zero_rois.append(key)
print("Frames with 0 ROIs:")

for frame in frames_with_zero_rois:
    print(frame)


In case you want to check the coords for each roi, you can use following cell to print it

In [ ]:
print("\nContents of roi_ellipse dictionary:")
for key, ellipses in roi_ellipse.items():
    print(f"Key: {key}, Number of ROIs: {len(ellipses)}")
    for i, ellipse in enumerate(ellipses):
        print(f"  ROI {i}: Center={ellipse.center}, Radii={ellipse.rad}")
        

In [ ]:
rois = roi_ellipse['AD23-097-s1-anosmic_concatenated']
time_in_roi = time_roi.TimeinRoi(fps=60)
file_path = '/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/24-02-ca2+img/AD23-097/anosmic/concatenated_files/AD23-097-s1-anosmic_concatenated.h5'

for roi in rois:
    time_in_roi.add_roi(roi)

tracking_data = time_in_roi.extract_tracking_data(file_path, scorer, bodypart)
time_in_rois = time_in_roi.time_in_rois(tracking_data)
print(time_in_rois)

Because each video might have slightly different fps, let's get a specific value for each video


In [ ]:
time_in_roi = time_roi.TimeinRoi(fps=None)
results_df = time_in_roi.time_in_rois_dir(directory=video_dir, 
                                          rois=roi_ellipse, 
                                          scorer=scorer, 
                                          body_part=bodypart, 
                                          file_endswith='_filtered.h5_centroid.h5', 
                                          filename_replace = 'DLC_resnet50_conductaJan11shuffle1_600000_filtered.h5_centroid.h5')
results_df.to_hdf('/Volumes/ANNA_HD/ANALYSIS/EXPERIMENTS/2024/22-09-sr/results.h5', key='df', mode='w')
print(results_df)